In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import json
import matplotlib.pyplot as plt
import sklearn
from sklearn.decomposition import TruncatedSVD

import warnings; warnings.simplefilter('ignore')
%matplotlib inline

In [2]:
orders_data=pd.read_csv("Orders.csv")
orders_data=orders_data.drop(columns=['Unnamed: 0'])
orders_data.head()

,customerID,orderID,Date,item
0,1808,2810,07/06/2021,Anand Ghee
1,1808,2810,07/06/2021,Ashirvaad Aata
2,1808,2810,07/06/2021,tea
3,1808,2810,07/06/2021,bottled water
4,1808,2810,07/06/2021,instant coffee


In [3]:
#Finding which day of month they bought what and then recommending items which were bought more than once.
orders_data['Day']=orders_data['Date'].str[0:2]
orders_data["Day"] = pd.to_numeric(orders_data["Day"])
cID=1808
d=5
    
df=pd.DataFrame(orders_data.loc[(orders_data['Day']>=d-5) & (orders_data['Day']<=d+5) & (orders_data['customerID']==cID)])
bool_series=df["item"].duplicated()
time_of_visit=df[bool_series]['item'].unique().tolist()
print(time_of_visit)

['shopping bags', 'Anand Ghee', 'tea', 'frozen fruits', 'instant coffee', 'waffles']


In [4]:
#Recommending items that were bought more than once.
new_df=pd.DataFrame(orders_data.loc[(orders_data['customerID']==cID)])
bool_s=new_df["item"].duplicated()
past_orders=new_df[bool_s]['item'].unique().tolist()
past_orders

['shopping bags',
 'Anand Ghee',
 'tea',
 'frozen fruits',
 'instant coffee',
 'waffles',
 'hamburger meat',
 'specialty fat',
 'Cornflakes']

In [5]:
#Recommending similar items 
count_products = pd.DataFrame(orders_data.groupby('item')['orderID'].count())
count_products = count_products.sort_values('orderID', ascending=False)
count_products.rename(columns = {'orderID':'count'}, inplace = True)
C = count_products['count'].sum()

def rating(x, C=C):
    v = x['count']
    return v*100/C

count_products['rating_calculated']=count_products.apply(rating, axis=1)

merged_inner = pd.merge(left=orders_data, right=count_products, left_on='item', right_on='item')

ratings_utility_matrix = merged_inner.pivot_table(values='rating_calculated', index='customerID', columns='item', fill_value=0)

X = ratings_utility_matrix.T

X1 = X

SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
correlation_matrix = np.corrcoef(decomposed_matrix)
i = "berries"

product_names = list(X.index)
item = product_names.index(i)

correlation_product_ID = correlation_matrix[item]

Recommend = list(X.index[correlation_product_ID > 0.65])

# Removes the item already bought by the customer
Recommend.remove(i) 
Recommend[0:24]



['Dhara refined Sunflower oil',
 'Maggie',
 'cleaner',
 'frozen dessert',
 'fruit/vegetable juice',
 'hamburger meat',
 'meat spreads',
 'mustard',
 'photo/film',
 'processed cheese',
 'specialty cheese',
 'specialty fat',
 'tea',
 'waffles']

In [6]:
#Calculating counts of each items bought by all customers and then sorting them in descending order.
#Most Bought products
most_popular = orders_data.groupby('item')['item'].count().sort_values(ascending=False)
most_popular.head(10)

item
Instant food products       24
frozen fish                 22
frozen fruits               22
long life bakery product    22
herbs                       21
hamburger meat              20
dish cleaner                17
specialty fat               17
waffles                     15
dog food                    14
Name: item, dtype: int64